In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/edgi-govdata-archiving/EEW-EJScreen.git
    %mv /content/EEW-EJScreen/ejscreen.db /content
else:
    print('Not running on CoLab')

In [ ]:
# Enter the state and CD. Then you can run all.
state = 'MA'
cd = '07'

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect( "ejscreen.db" )

sql = 'select FIPS_Code from states where State_Code=\'{}\''.format(state)
df = pd.read_sql_query(sql, conn)
state_fips = df['FIPS_Code'][0].zfill(2)


In [ ]:
select_columns = 'P_LDPNT_D2, P_DSLPM_D2, P_CANCR_D2, P_RESP_D2, P_PTRAF_D2, P_PWDIS_D2, '
select_columns += 'P_PNPL_D2, P_PRMP_D2, P_PTSDF_D2, P_OZONE_D2, P_PM25_D2'

sql = 'select {} from ejscreen where substr(id,1,2) = \'{}\' and '
sql += 'substr(id,6,2) = \'{}\''
sql = sql.format(select_columns, state_fips,cd)
df = pd.read_sql_query(sql, conn)
sql


In [ ]:
columns = ['P_LDPNT_D2', 'P_DSLPM_D2', 'P_CANCR_D2', 'P_RESP_D2', 'P_PTRAF_D2', 'P_PWDIS_D2', 
           'P_PNPL_D2', 'P_PRMP_D2', 'P_PTSDF_D2', 'P_OZONE_D2', 'P_PM25_D2']
df[columns] = df[columns].apply(pd.to_numeric)
means = pd.DataFrame(df.mean())
new_columns=['Lead paint', 'Diesel', 'Air toxics cancer', 'Air toxics resp',
                                      'Traffic', 'Water discharge', 'NPL sites', 'RMP facilities',
                                      'TSDF facilities', 'Ozone', 'PM2.5']
means = means.set_axis(new_columns, axis=0)

stdev = pd.DataFrame(df.std())
stdev = stdev.set_axis(new_columns, axis=0)


In [ ]:
df.to_csv('ejscreen-{}{}-percentiles.csv'.format(state, cd))

In [ ]:
df2 = pd.melt(df)
df2['value2'] = pd.to_numeric(df2['value'], errors='ignore')
df2['bins'] = pd.cut(df2['value2'], bins=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95])

In [ ]:
newdf = pd.DataFrame(columns=['metric', 'value', 'count'])
for metric in columns:
    tempdf = df2[df2['variable'] == metric]
    for bin in tempdf['bins'].unique():
        try:
            midpoint = (bin.left + bin.right)/2
            tempdf2 = tempdf[tempdf['bins'] == bin]
            count = 2 * tempdf2['value'].count()
            new_row = {'metric':metric, 'value':midpoint, 'count':float(count)}
            # breakpoint()
            newdf = newdf.append(new_row,ignore_index=True)
        except AttributeError:
            continue
newdf       
    

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [8, 5]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

means.plot(kind='bar', alpha=0., yerr = stdev)
# means_plus_stdev.plot(kind='bar', alpha=0.3)
plt.scatter(x=newdf['metric'], y=newdf['value'], sizes=newdf['count'],
           color='Green')

plt.title("EJ Screen Index Percentiles for {}, District {}".format(state,cd))
plt.xlabel("EJ Screen Categories")
plt.ylabel("Census Blocks in National Percentiles\n and Standard Deviation")
ax = plt.gca()
ax.set_xticklabels(labels=new_columns,rotation=60)
# plt.ylim(0,100)
# plt.axhline(y = 50, color = 'r', linestyle = '-')
filename = "graph-{}-{}.png".format(state, cd)
plt.savefig(filename, bbox_inches="tight")

In [ ]:
xdf = pd.read_csv('ej_screen_cd_stdev_rankings.csv')
ydf = xdf[['Lead_Paint.Pct','Diesel.Pct','Air_toxics_cancer.Pct','Air_toxics_resp.Pct',
                   'Traffic.Pct','Water_discharge.Pct','NPL_sites.Pct','RMP_facilities.Pct',
                  'TSDF_facilities.Pct','Ozone.Pct','PM2.5.Pct','State','CD']]
zdf = ydf.loc[ydf['State'] == state]
rank_df = zdf.loc[zdf['CD'] == int(cd)]

In [ ]:
import numpy as np

grades = ['A','B','C','D','F']

columns = list(rank_df)
for i in columns:
    nm = rank_df[i].name
    if nm != 'State' and nm != 'CD':
        s = rank_df[i]
        value = s[s.index[0]]
        grade = 'None'
        if np.issubdtype(rank_df[i].dtype, np.number):
            value = int(value/0.2)
            grade = grades[value]
        print('{} - Grade for "fairness" in the district is {} ({} percentile nationally)'.format(
            nm, grade, int(s[s.index[0]]*100)))
